## The data dictionary provided with data is as follows:

- dateCrawled - When this ad was first crawled. All field-values are taken from this date.
- name - Name of the car.
- seller - Whether the seller is private or a dealer.
- offerType - The type of listing
- price - The price on the ad to sell the car.
- abtest - Whether the listing is included in an A/B test.
- vehicleType - The vehicle Type.
- yearOfRegistration - The year in which the car was first registered.
- gearbox - The transmission type.
- powerPS - The power of the car in PS.
- model - The car model name.
- kilometer - How many kilometers the car has driven.
- monthOfRegistration - The month in which the car was first registered.
- fuelType - What type of fuel the car uses.
- brand - The brand of the car.
- notRepairedDamage - If the car has a damage which is not yet repaired.
- dateCreated - The date on which the eBay listing was created.
- nrOfPictures - The number of pictures in the ad.
- postalCode - The postal code for the location of the vehicle.
- lastSeenOnline - When the crawler saw this ad last online.
The aim of this project is to clean the data and analyze the included used car listings. You'll also become familiar with some of the unique benefits jupyter notebook provides for pandas.

Let's start by importing the libraries we need and reading the dataset into pandas.

In [ ]:
import pandas as pd
import numpy as np
autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [ ]:
autos

In [ ]:
autos.info()
autos.head()

&#9642 The data contains 20 columns. Most of them are strings, others are integers. It might be able to transfer strings to numbers.
&#9642 Some columns have null values, but none have more than ~20% null values.

### Clean data Step 1-Modify column names

In [ ]:
print(autos.columns)

In [ ]:
autos.columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen']

In [ ]:
autos.head()

### Clean data Step 2 - Basic data exploration

In [ ]:
autos.describe(include='all')

 Drop seller, offer_type (mostly one value)
 Nr_of_pictures needs more investigation
 price; odometer; ad_created; are text that need to be cleaned

In [ ]:
autos.drop(["seller", "offer_type"], axis = 1 , inplace = True)
autos["odometer"] = (autos["odometer"]
                    .str.replace("km", "")
                    .str.replace(",", "")
                    .astype(float))
autos["price"] = (autos["price"]
                  .str.replace("$","")
                  .str.replace(",","")
                  .astype(float))
autos.rename(columns = {"odometer":"odometer_km"}, inplace=True)

In [ ]:
print(autos["odometer_km"].unique().shape)
print(autos["price"].unique().shape)

In [ ]:
print(autos[['odometer_km', 'price']].describe())

Remove outliers for price too high (>999900) and too low (=0)

In [ ]:
autos['price'].value_counts().sort_index(ascending=False).head(30)
autos= autos[autos['price'].between(0,999990)]

In [ ]:
autos['odometer_km'].value_counts()

There are 5 columns that should represent date values. Some of these columns were created by the crawler, some came from the website itself. We can differentiate by referring to the data dictionary:


- `date_crawled`: added by the crawler
- `last_seen`: added by the crawler
- `ad_created`: from the website
- `registration_month`: from the website
- `registration_year`: from the website
Right now, the date_crawled, last_seen, and ad_created columns are all identified as string values by pandas. Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can understand it quantitatively. The other two columns are represented as numeric values, so we can use methods like Series.describe() to understand the distribution without any extra data processing.

Let's first understand how the values in the three string columns are formatted. These columns all represent full timestamp values, like so:


autos[['date_crawled','ad_created','last_seen']][0:5]
date_crawled	ad_created	last_seen
0	2016-03-26 17:47:46	2016-03-26 00:00:00	2016-04-06 06:45:54
1	2016-04-04 13:38:56	2016-04-04 00:00:00	2016-04-06 14:45:08
2	2016-03-26 18:57:24	2016-03-26 00:00:00	2016-04-06 20:15:37
3	2016-03-12 16:58:10	2016-03-12 00:00:00	2016-03-15 03:16:28
4	2016-04-01 14:38:50	2016-04-01 00:00:00	2016-04-01 14:38:50
You'll notice that the first 10 characters represent the day (e.g. 2016-03-12). To understand the date range, we can extract just the date values, use Series.value_counts() to generate a distribution, and then sort by the index.

In [ ]:
(autos["date_crawled"]
 .str[:10]
 .value_counts(normalize=True, dropna=False)
 .sort_index())

In [ ]:
print(autos["ad_created"]
 .str[:10]
 .value_counts(normalize=True, dropna=False)
 .sort_index())
print(autos["last_seen"]
 .str[:10]
 .value_counts(normalize=True, dropna=False)
 .sort_index())

Count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.
Decide which the highest and lowest acceptable values are for the registration_year column.

In [ ]:
registration_year = autos[autos["registration_year"].between(1900, 2018)]

In [ ]:
(registration_year["registration_year"]
 .value_counts(normalize=True)
 .sort_index())

The distribution shows more car are in between 1999 and 2010

In [ ]:
# brands = autos["brand"].value_counts(normalize=True).sort_index(ascending=False).index
# brands
brand_counts = autos["brand"].value_counts(normalize=True)
brands = brand_counts[brand_counts > .05].index
print(brands)

In [ ]:
brands_ag = dict()
for brand in brands:
    brandonly = autos[autos["brand"]==brand]
    brands_ag[brand]= brandonly["price"].mean()

In [ ]:
brands_ag

` Audi, BMW and Mercedes Benz are more expensive
` Ford and Opel are less expensive
` Volkswagen is in between

In [ ]:
# Another way to get the highest 6 brand
brands = (autos["brand"]
          .value_counts(normalize=True)
          .sort_values(ascending=False)
          .head(6).index)
brands

In [ ]:
brands_milage = dict()
for brand in brands:
    brandonly = autos[autos["brand"]==brand]
    brands_milage[brand]= brandonly["odometer_km"].mean()

In [ ]:
brands_milage

In [ ]:
auto_series = pd.Series(brands_ag)
print(auto_series)
df=pd.DataFrame(auto_series, columns=['mean_price'])
print(df)

In [ ]:
df["mean_milage"]=brands_milage
print(df)